In [1]:
import sqlite3
import pandas as pd
from scipy.stats.stats import pearsonr
from datetime import datetime
import time
import numpy as np

In [17]:
df_rf  = pd.read_csv('rf.csv')
time_list = df_rf['y/m'].tolist()
rf_list = [float(x)/5200 for x in df_rf['rf'].tolist()]

time_to_rf = dict()
for i in range(len(time_list)):
    year, month = time_list[i].split('/')
    year = int(year) + 1911
    month = int(month)
    string = str(year) +'-%02d'%month #%Y-%m
    time_to_rf[string] = rf_list[i]

In [12]:
db = sqlite3.connect('fund.sqlite')
cur = db.cursor()

In [41]:
cur.execute('PRAGMA  table_info([week])')

In [42]:
rf_array = []
r = cur.fetchall()
for i in range(1, len(r)):
    date = r[i][1][:-3]
    rf_array.append( time_to_rf[date]  )
rf_array = np.array(rf_array)

In [43]:
rf_array

array([0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019904,
       0.00019904, 0.00019904, 0.00019904, 0.00019904, 0.00019

In [44]:
cur.execute('select * from week WHERE "index" = "F0000112ZA:FO"')
r = cur.fetchall()

x = np.array(r[0][1:], dtype= 'float64')
x_new = x[~np.isnan(x)]


In [47]:
rf_array_new = rf_array[~np.isnan(x)]

In [49]:
x_new - rf_array_new

array([-0.00019904, -0.00119904, -0.00019904, -0.00019904, -0.00220104,
        0.00180698,  0.00380497,  0.00179498, -0.00119406,  0.00079698,
       -0.00019904, -0.00417914,  0.00479597,  0.00576518,  0.00671796,
        0.00370721, -0.00214456,  0.0036996 ,  0.00368446,  0.00753791,
        0.00076065, -0.00211658, -0.0107658 ,  0.00174271,  0.00464592,
       -0.00116336, -0.00019904])

# output => 輸入基金名 返回相關係數跟報酬率 (已經減去rf)

In [53]:
t1 = time.time()

cor_list = []
id_list = []
mean_list = []
db = sqlite3.connect('fund.sqlite')
cur = db.cursor()


# get rf
df_rf  = pd.read_csv('rf.csv')
time_list = df_rf['y/m'].tolist()
rf_list = [float(x)/5200 for x in df_rf['rf'].tolist()]

time_to_rf = dict()
for i in range(len(time_list)):
    year, month = time_list[i].split('/')
    year = int(year) + 1911
    month = int(month)
    string = str(year) +'-%02d'%month #%Y-%m
    time_to_rf[string] = rf_list[i]
    
cur.execute('PRAGMA  table_info([week])')
rf_array = []
r = cur.fetchall()
for i in range(1, len(r)):
    date = r[i][1][:-3]
    rf_array.append( time_to_rf[date]  )
rf_array = np.array(rf_array)




# 搜尋基金
cur.execute('select * from week WHERE "index" = "F0000112ZA:FO"')
r = cur.fetchall()

x = np.array(r[0][1:], dtype= 'float64')

rf_array = rf_array[~np.isnan(x)]
x_new = x[~np.isnan(x)] - rf_array




cur.execute('select * from week WHERE "index" != "F0000112ZA:FO"')
print('資料載完花了', (time.time()-t1))


r = cur.fetchall()
for i in range(len(r)):
    id_list.append(r[i][0])
    y = np.array(r[i][1:], dtype= 'float64')[~np.isnan(x)] - rf_array
    mean_list.append(y.mean()*4)
    cor,_ = pearsonr(x_new, y)
    cor_list.append(cor )
    
a_array = np.array(cor_list)
indices = a_array.argsort()[:6]

print('Finished', (time.time()-t1))
print('F0000112ZA:FO', 1, x_new.mean())

output = []


for i in indices[0:3]: 
    output.append( (mean_list[i], 'A+', id_list[i], a_array[i])  )
for i in indices[3:]: 
    output.append( (mean_list[i], 'A', id_list[i], a_array[i])  )
    
output.sort(reverse = True)
print('mean, rank, a_id, cor')
for mean, rank, a_id, cor in output:
    print(mean, rank, a_id, cor)

資料載完花了 0.01562047004699707
Finished 0.5701982975006104
F0000112ZA:FO 1 0.0009371362732923888
mean, rank, a_id, cor
0.005541986956028405 A F0GBR06IGW:FO -0.35006388135963834
0.0022396425055559244 A F0GBR06TV0:FO -0.34180529904724033
0.0006674413133922558 A FOGBR05KMB:FO -0.34626952175978876
-0.0005760227467177637 A+ F0HKG05WX5:FO -0.5254068110564654
-0.0005999310473159895 A+ F0HKG05WVT:FO -0.4117816993623376
-0.0006105044648142572 A+ F0HKG05X0R:FO -0.5254491835021496
